<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам

По условию соревнования, нам нужно самостоятельно собрать обучающий датасет. 

## Парсер сайта AUTO.RU (Актуальность: декабрь 2020 г.)

In [ ]:
"""
from fake_useragent import UserAgent 
from bs4 import BeautifulSoup    
import requests  
import re
import time
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm.notebook import tqdm




# Функция по извлечению данных со страницы объявления в словарь data_dict
def parsing_page_one_ad(url):

    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})    
    response.encoding ='utf8'   
    
    # Теперь создадим объект BeautifulSoup, указывая html парсер    
    page = BeautifulSoup(response.text, 'html.parser')
    data_dict = {}

    data_dict['car_url'] = url
    data_dict['parsing_unixtime'] = int(time.time())


     # в разделах script ищем вхождение 'complectation":{"id"'
    for script in page.find_all("script"):
        if 'complectation":{"id"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
    # в a  ищем 'complectation":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]

        if 'equipment":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем 'equipment":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]

        if '{"mileage":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
        # в a  ищем '{"mileage":' и содержимое между {}. Отсекаем начало, оставляя только содержимое словаря
            data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][10:]

        if '"model_info":' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')

        if 'super_gen":{' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:] 

        if 'vendor":"' in str(script):
            a = str(script)  # присваиваем a содержимое скрипта как строку
            data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')


    for tag in page.find_all('div'):
        if tag.get("title") == "Идентификатор объявления":
            data_dict['sell_id'] = re.search(r'\d+', tag.text)[0]


    for tag in page.find_all("meta"):
        if tag.get("itemprop") == "bodyType":
            data_dict['bodyType'] = tag.get("content")

        if tag.get("itemprop") == "brand":
            data_dict['brand'] = tag.get("content")

        if tag.get("itemprop") == "color":
            data_dict['color'] = tag.get("content")

        if tag.get("itemprop") == "description":
            data_dict['description'] = tag.get("content")

        if tag.get("itemprop") == "engineDisplacement":
            data_dict['engineDisplacement'] = tag.get("content")

        if tag.get("itemprop") == "enginePower":
            data_dict['enginePower'] = tag.get("content")

        if tag.get("itemprop") == "fuelType":
            data_dict['fuelType'] = tag.get("content")

        if tag.get("itemprop") == "modelDate":
            data_dict['modelDate'] = tag.get("content")

        if tag.get("itemprop") == "name":
            data_dict['name'] = tag.get("content")

        if tag.get("itemprop") == "numberOfDoors":
            data_dict['numberOfDoors'] = tag.get("content")

        if tag.get("itemprop") == "price":
            data_dict['price'] = tag.get("content")

        if tag.get("itemprop") == "priceCurrency":
            data_dict['priceCurrency'] = tag.get("content")

        if tag.get("itemprop") == "productionDate":
            data_dict['productionDate'] = tag.get("content")

        if tag.get("itemprop") == "vehicleConfiguration":
            data_dict['vehicleConfiguration'] = tag.get("content")

        if tag.get("itemprop") == "vehicleTransmission":
            data_dict['vehicleTransmission'] = tag.get("content")


    span_CardInfoRow__cell = page.find_all('span', {'class': 'CardInfoRow__cell'})

    for i,tag in enumerate (span_CardInfoRow__cell):
        if tag.text == "Владельцы":
            data_dict['Владельцы'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Владение":
            data_dict['Владение'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "ПТС":
            data_dict['ПТС'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел    

        if tag.text == "Привод":
            data_dict['Привод'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Руль":
            data_dict['Руль'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

        if tag.text == "Состояние":
            data_dict['Состояние'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел 

        if tag.text == "Таможня":
            data_dict['Таможня'] = span_CardInfoRow__cell[i+1].text.replace(u'\xa0', u' ') # в  конце заменяем юникодовский пробел

    return data_dict

    
# Функция по созданию списка ссылок links_list  на объявления о продаже автомобилей  
def extraction_links(url):
    links_list =[] 
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})  
    response.encoding ='utf8'
    page = BeautifulSoup(response.text, 'html.parser') 
    links = page.find_all('a', class_='Link ListingItemTitle-module__link')
    
    for link in links:
        links_list.append(link.get("href"))
    return links_list
    
    
    
    
    
df = pd.DataFrame() # инициализируем итоговый датафрейм 

url_link_list = [] # список страниц по годам и номерам от 1 до 99

ranges = [range(2006, 2011), range(1, 100)] # указываем года и диапазон страниц, которые будем парсить 

# index[0] - year, index[1] - page 
for index in itertools.product(*ranges):
    # формируем ссылки страницы со списками объявлений
    на  = (f"https://auto.ru/moskva/cars/{index[0]}-year/all/?output_type=table&page={index[1]}")
    url_link_list.append(url_links) # заносим их в список
    
links_list = [] # список списков ссылок на объявления c одной страницы таблицы объявлений

#for url_links in url_link_list:
# извлекаем в links_list список ссылок на объявления
try:
        #links_list = extraction_links(url_links)
    links_list = Parallel(n_jobs = 2)(delayed(extraction_links)(url_links) for url_links in url_link_list)
except:
    pass 

ads_dict_list = [] # список словарей содержимого объявлений

for links in links_list:
    try:
        ads_dict_list = Parallel(n_jobs = 2)(delayed(parsing_page_one_ad)(ad_url) for ad_url in links)
    except:
        pass
    
    for ad in ads_dict_list:
        try:
            df = df.append(ad, ignore_index=True)
        except:
            pass

df.to_csv('_auto_ru_XXXX-XXXX.csv', encoding = 'utf-8', index=False) # записываем содержимое датафрейма в файл
"""

## Объединение спарсенных датафреймов в один и его первичная обработка

In [ ]:
"""
import os
import pandas as pd
import numpy as np

df = pd.DataFrame()
list_file = os.listdir(path=".") # получение списка файлов в папке

for file in list_file:
    if '.csv' in file: # если у файла расширение .csv, то читаем его как DF
        df_temp = pd.read_csv(file)
        df = pd.concat([df, df_temp], ignore_index=True)
        
df = df.drop_duplicates(subset=['car_url']) # удаление дубликатов по URL объявления

df = df.replace('{}',  None) # замена пустых словарей на NaN
df['mileage'] = df['mileage'].apply(lambda x: int(x) if type(x) == str else x) # приведение 'mileage' к числовму виду
df = df.replace(np.nan,  None)

df.to_csv('auto_ru_moskva_12_12_2020.csv', encoding = 'utf-8', index=False)
"""

Подгружаем библиотеки и наш датасет

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import seaborn as sns
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

In [ ]:
pd.set_option('display.max_columns', None) #отображение всех столбцов датасета

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

### Определение базовых функций

In [ ]:
# Функция по сбору информации по каждому столбцу


def my_describe(df):
    """Отображение описательных статистик датафрейма в удобной форме"""
    temp = {}
    temp['Имя признака'] = list(df.columns)
    temp['Тип'] = df.dtypes
    temp['Всего значений'] = df.describe(include='all').loc['count']
    temp['Число пропусков'] = df.isnull().sum().values 
    temp['Кол-во уникальных'] = df.nunique().values
    temp['Минимум'] = df.describe(include='all').loc['min']
    temp['Максимум'] = df.describe(include='all').loc['max']
    temp['Среднее'] = df.describe(include='all').loc['mean']
    temp['Медиана'] = df.describe(include='all').loc['50%']
    temp = pd.DataFrame.from_dict(temp, orient='index')
    display(temp.T)
    return


def show_plot_boxplot(df, column, bins=80):
    """Построение гистограммы по столбцу и boxplot-а"""
    color_text = plt.get_cmap('PuBuGn')(0.9)
    plt.style.use('seaborn')
    plt.rcParams['figure.figsize'] = (10, 4)
    _, axes = plt.subplots(2, 1)
    axes[0].hist(df[column], bins=bins)
    axes[0].set_title("Гистограмма и boxplot для признака "+column)
    axes[1].boxplot(df[column], vert=False, showmeans = True)
    axes[1].set_title('')
    return


# Пропишем функцию расчета IQR и квартилей.

def IQR_perc(df,list_cols):
    temp = {}
    temp['Имя признака'] = list_cols
    temp['IQR'] = df[list_cols].quantile(0.75) - df[list_cols].quantile(0.25)
    temp['perc25'] = df[list_cols].quantile(0.25)
    temp['perc75'] = df[list_cols].quantile(0.75)
    temp['Л. граница выбросов'] = df[list_cols].quantile(0.25) - 1.5*(df[list_cols].quantile(0.75) - df[list_cols].quantile(0.25))
    temp['П. граница выбросов'] =df[list_cols].quantile(0.75) + 1.5*(df[list_cols].quantile(0.75) - df[list_cols].quantile(0.25))
    temp = pd.DataFrame.from_dict(temp, orient='index')
    display(temp.T)
    return


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true)) # функция вычисления MAPE – средней абсолютной ошибки

# Setup

In [ ]:
VERSION    = 15
DIR_TRAIN = '/kaggle/input/autorumoskva/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '/kaggle/input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'auto_ru_moskva_12_12_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.sample(5)

In [ ]:
my_describe(train)

In [ ]:
plt.figure(figsize=(6, 6))
plt.ylim(top=3_000_000)
ax = sns.boxplot(y="price", data=train)

> На графике видим, что 75% предложений, это цена до 1_300_000р. И все что свыше  3 млн. рублей - считается выбросами 

In [ ]:
len(train[train['price']>3_000_000])

In [ ]:
print(f'Общая доля выбросов по признаку "price" :', round(len(train[train['price']>3_000_000])/len(train) * 100,2), '%')

In [ ]:
test.head(5)

In [ ]:
my_describe(test)

In [ ]:
# В train отсутствует признак Image, поэтому в test сразу его удалим
test.drop(['image'], axis=1,inplace=True)

## Data Preprocessing

In [ ]:
# сохраним целевую переменную, а после удалим из тренировочного датасета
y = train['price']

In [ ]:
train.drop(['price'], axis=1,inplace=True)

In [ ]:
df_train = train#[columns]
df_test = test#[columns]

# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

### 1. Первичный отсмотр данных

In [ ]:
my_describe(data)

In [ ]:
#Удалим столбцы не несущие в себе параметры автомобиля, и не влиящие на предсказание цены 
data.drop(['car_url', 'parsing_unixtime', 'sell_id'], axis=1,inplace=True)
# В связи с отсутствие времени на обработку текста, также удалим 'description'
data.drop(['description'], axis=1,inplace=True)

### Работа с пропусками

In [ ]:
# построим карту пропусков данных
f, ax = plt.subplots(figsize=(12, 5))
sns.heatmap(data.isnull(), cbar=True)
data.isnull().sum()

У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, даже отсутствие информации может быть важным признаком! По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак

In [ ]:
# Подсчитаем общее число строк с пропусками
data.shape[0] - data.dropna().shape[0]

In [ ]:
#пропуски в значительном количестве наблюдаем о в столбцах "complectation_dict",'equipment_dict' и 'Владение'.
#Создадим новые столбцы с признаком пропуска данных в них
cols ={"complectation_dict",'equipment_dict', 'Владение'}
for col in cols:
    data[f'{col}_isNAN'] = pd.isna(data[col]).astype('uint8')

In [ ]:
# один пропуск наблюдаем в "ПТС". Заполним его самым частым значением: data["ПТС"].value_counts().index[0]
data["ПТС"] = data["ПТС"].fillna(data["ПТС"].value_counts().index[0])

#### Признак "complectation_dict"

In [ ]:
data["complectation_dict"].value_counts()

> Самым частым непустым значением в столбце является значение {"id":"0"}. Чтобы не нарушать структуру данных оставим его без изменений, и им заполним пропуски в столбце. И ячейкам в  ['complectation_dict_isNAN'], соответствующих этому значению, присвоим единицу 

In [ ]:
data["complectation_dict"] = data["complectation_dict"].fillna('{"id":"0"}')
data['complectation_dict_isNAN'] = data["complectation_dict"].apply(lambda x: 1 if x == '{"id":"0"}' else 0)
data['complectation_dict_isNAN'] = data['complectation_dict_isNAN'].astype('uint8')

#### Признак "equipment_dict"

In [ ]:
data["equipment_dict"].value_counts()

In [ ]:
# Наиболее универсального варианта здесь не видно, поэтому присвоим пропускам значение : {"unknown" :true}

In [ ]:
data["equipment_dict"] = data["equipment_dict"].fillna('{"unknown" :true}')

#### Признак "Владение"

In [ ]:
  data["Владение"].value_counts()  

In [ ]:
data['Владельцы'][data["Владение"].isnull()].value_counts()

Так как  машины с пропусками в признаке "Владение" выпущены в разные годы (большинство не новые из автосалона), и зачастую имели более одного владельца, то пропуски заполним значением: 'Неизвестно'  

In [ ]:
data["Владение"] = data["Владение"].fillna('Неизвестно')

In [ ]:
my_describe(data)

In [ ]:
# Просматриваем и обрабатываем все признаки

## Обработка признаков

Для начала посмотрим какие признаки у нас могут быть категориальныe, бинарные и числовые:

In [ ]:
data.nunique(dropna=False)

In [ ]:
# сформируем списки столбцов по группам исходя из типов признаков
num_cols = ['mileage', 'modelDate', 'productionDate']
bin_cols =['Руль', 'Состояние', 'Таможня', 'ПТС'] 
category_cols = ['model_info', 'model_name', 'name', 'super_gen', 'complectation_dict', 'equipment_dict',  'bodyType', 'brand', 'color', 'fuelType', 'numberOfDoors', 
                 'priceCurrency',  'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'Привод', 'engineDisplacement', 'enginePower']
# 'priceCurrency' - тип валюты. В настоящее время у нас только рубли, но могут быть и др. варианты. Поэтому отнесем к категориальным данным 

### Числовые признаки

In [ ]:
# посмотрим параметры IQR, гистограммы и боксплоты числовых признаков
IQR_perc(data, num_cols)
for column in num_cols:
    show_plot_boxplot(data,column)

Выводы:

По графикам и предыдущим таблицам видно, что основное число машин  произведено примерно в 2010г и в среднем имеют пробег 146_000 км. Данные разрежены, позднее логарифмируем их

Попытка удаления выбросов в тренировочном датасете метрику не улучшило

### Корреляция

In [ ]:
# Для проверки корреляции возьмем числовые признаки тренировочного датасета 
# и присоединим к ним целевую переменную в виде датафрейма df
X = data.query('sample == 1').drop(['sample'], axis=1)

df = pd.DataFrame(X[num_cols]).reset_index()
df.drop(['index'], axis=1,inplace=True)
df['price']=y
df

In [ ]:
correlation = df.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

Видим, что modelDate и ProductionDate как и ожидалось сильно коррелируют между собою, а также  engineDisplacement и enginePower слабо  коррелируют с целевой переменной (ценой). Пробег же с целевой переменной корреллирует отрицательно - чем больше пробег, тем ниже цена

### Бинарные признаки

In [ ]:
# Переведем бинарные признаки в числа при  LabelEncoder

label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    
# убедимся в преобразовании    
data.head()

### Категориальные признаки

Осмотрим значения признаков имеющих неболшое количество значений: 

In [ ]:
for col in ['bodyType', 'color', 'engineDisplacement', 'enginePower', 'fuelType', 'numberOfDoors', 'priceCurrency', 
'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'Привод']:
    print(f'{col}: {data[col].unique()}')

In [ ]:
# В признаке 'Владельцы' заменим юникодовский пробел на ascii пробел
data["Владельцы"] = data["Владельцы"].str.replace(u'\xa0', '')

In [ ]:
# Переведем категориальные признаки в числа
# в алфавитном порядке 
"""label_encoder = LabelEncoder()

for column in category_cols:
    data[column] = label_encoder.fit_transform(data[column])
data.head()"""

for colum in category_cols:
    data[colum] = data[colum].astype('category').cat.codes

## Создание новых признаков

In [ ]:
data['km_in_year'] = data['mileage'] / ((datetime.datetime.now().year +1) - data['productionDate'])
num_cols.append('km_in_year')

# Введение данных признаков (атикварные и старые автомобили ) немного ухудшило метрику

#data['antique_car'] = data['productionDate'].apply(lambda x: 1 if x<(datetime.datetime.now().year-50) else 0)
#data['old_car'] = data['productionDate'].apply(lambda x: 1 if \
#((datetime.datetime.now().year-50)<= x <=(datetime.datetime.now().year-20)) else 0)

In [ ]:
# Преобразуем числовые признаки в полиномиальные
poly = PolynomialFeatures(3,interaction_only=True)
data_num = pd.DataFrame(poly.fit_transform(data[num_cols].values))

### Логарифмирование признаков (числовых и целевого)

In [ ]:
# Прологарифмируем целевую функцию. А после предсказания взять экспоненту от получившегося рузультата: y=np.exp(y)-1
y= np.log(y+1)
# прологарифмируем числовые признаки и полученные от них полиномиальные признаки
data[num_cols] = np.log(data[num_cols]+1)
data_num = np.log(data_num +1)

In [ ]:
# Присоединим к датасету ,блок нормализованных полиномиальных признаков
data = pd.concat([data,data_num], axis=1)

In [ ]:
# отделим тренировочный датасет от тестового:

X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

## Подготовка данных к машинному обучению

Разделим  тренировочные данные для обучения следующим образом:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Модели ML

## Model #1: Модель CatBoost c заданными параметрами 


![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

In [ ]:
# вычисления закомментированы для ускорения записи на Kaggle
model1 = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
#model1.fit(X_train, y_train,
#         eval_set=(X_test, y_test),
#         verbose_eval=100,
#         use_best_model=True,
#         plot=True
#         )

In [ ]:
predict = np.round((np.exp(model1.predict(X_test))-1)/1000) *1000
y_test_exp = np.exp(y_test)-1

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test_exp, predict))*100:0.2f}%")

Без подбора параметров MAPE составила 16.93% 

In [ ]:
# Не знаю пока что это, но сохраню на будущее
#model1.save_model('catboost_single_model_baseline.model')

### Модель 2 (с найденными лучшими гиперпараметрами)

In [ ]:
# Поиск наилучших параметров для CatBoostRegressor
cbr = CatBoostRegressor()

grid = {'learning_rate': [round(x,3) for x in np.linspace(start = 0.025, stop = 0.4, num = 40)],
        'depth': [int(x) for x in np.linspace(start = 5, stop = 10, num = 5)],
        'l2_leaf_reg': [int(x) for x in np.linspace(start = 1, stop = 5, num = 5)]}

#grid_search_result = cbr.grid_search(grid, 
#                                       X=X_train, 
#                                       y=y_train, 
#                                       plot=False)
#print(grid_search_result['params'])

# {'depth': 7, 'l2_leaf_reg': 1, 'learning_rate': 0.083} bestTest = 0.01254271153 bestIteration = 3746

In [ ]:
model2 = CatBoostRegressor(iterations = 3747,
                          learning_rate = 0.083,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          depth = 7,
                          l2_leaf_reg = 1 
                         )
#model2.fit(X_train, y_train,
#         eval_set=(X_test, y_test),
#         verbose_eval=100,
#         use_best_model=True,
#         plot=True
#         )

In [ ]:
#predict = np.round((np.exp(model2.predict(X_test))-1)/1000) *1000#
#y_test_exp = np.exp(y_test)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 16.97%
Модель с лучшими параметрами  на  X_sub(test) улучшила метрику на 0.5%

### Model 3. Стекинг

Применим в стекинге  sklearn.StackingRegressor() следующие алгоритмов:  CatBoostRegressor, LinearRegression, KNeighborsRegressor, GradientBoostingRegressor и RandomForestRegressor. И для некоторых попробуем найти оптимальные значения

Подбор параметров для RandomForest  

In [ ]:
# вычисления закомментированы
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}
#rf = RandomForestRegressor(random_state = RANDOM_SEED)
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=RANDOM_SEED, n_jobs = -1)
#rf_random.fit(X_train, y_train)
#rf_random.best_params_

#best_params_: 
#{'n_estimators': 200,
 #'min_samples_split': 2,
 #'min_samples_leaf': 2,
 #'max_features': 'auto',
# 'max_depth': None,
# 'bootstrap': True}

In [ ]:
rf = RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=200, min_samples_split=2, min_samples_leaf=2, 
                            max_features='auto', max_depth=None, bootstrap=True)
#rf.fit(X_train, y_train)

#predict2 = np.round((np.exp(rf.predict(X_test))-1)/1000) *1000
#y_test_exp2 = np.exp(y_test)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp2, predict2))*100:0.2f}%")

Точность модели по метрике MAPE: 17.56%

Подбор параметров для KNeighborsRegressor

In [ ]:
#knr = KNeighborsRegressor()

#cv = 5
#grid = {'n_neighbors': [int(x) for x in np.linspace(start = 3, stop = 7, num = 3)],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'leaf_size': [int(x) for x in np.linspace(start = 5, stop = 50, num = 10)]}

#grid_search_result = GridSearchCV(knr, param_grid=grid, cv=cv).fit(X_train, y_train)

#print(grid_search_result.best_params_)
#{'algorithm': 'auto', 'leaf_size': 40, 'n_neighbors': 7, 'weights': 'distance'}

In [ ]:
knr = KNeighborsRegressor(algorithm='auto', leaf_size=40, n_neighbors=7, weights='distance', n_jobs=-1)
#knr.fit(X_train, y_train)

#predict3 = np.round((np.exp(knr.predict(X_test))-1)/1000) *1000
#predict = np.exp(model.predict(X_test))-1
#y_test_exp3 = np.exp(y_test)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp3, predict3))*100:0.2f}%")

Точность модели по метрике MAPE: 48.45%

Подбор параметров для GradientBoostingRegressor провести не удалось в связи с большей  длительностью данного процесса. Поэтому возьмем параметры по умолчанию, установим только n_estimators=300

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=300, random_state=RANDOM_SEED,  warm_start=10,  tol=0.0001)
#gbr.fit(X_train, y_train)

#predict5 = np.round((np.exp(gbr.predict(X_test))-1)/1000) *1000
#y_test_exp5 = np.exp(y_test)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp5, predict5))*100:0.2f}%")

Точность модели по метрике MAPE: 21.16%

### Стекинг. 

1 вариант

In [ ]:
# вычисления закомментированы для ускорения записи на Kaggle
estimators=[
        ('rf1', RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=200, min_samples_split=2, min_samples_leaf=2, 
                            max_features='auto', max_depth=5, bootstrap=True, n_jobs=-1)),
        ('gbr', GradientBoostingRegressor(n_estimators=300, random_state=RANDOM_SEED,  warm_start=10,  tol=0.0001)),
        ('knr', KNeighborsRegressor(algorithm='auto', leaf_size=40, n_neighbors=7, weights='distance', n_jobs=-1)),
            ]

reg1 = StackingRegressor(estimators=estimators,
     final_estimator = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = 0.083,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          depth=7,
                          l2_leaf_reg=1               
                        )
                    )

    
#reg1.fit(X, y)

#predict = np.round((np.exp(reg1.predict(X))-1)/1000) *1000
#y_test_exp = np.exp(y)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 26.05%

2 вариант

In [ ]:
# вычисления закомментированы для ускорения записи на Kaggle
estimators=[
        ('rf1', RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=200, min_samples_split=2, min_samples_leaf=2, 
                            max_features='auto', max_depth=5, bootstrap=True, n_jobs=-1)),
        ('gbr', GradientBoostingRegressor(n_estimators=300, random_state=RANDOM_SEED,  warm_start=10,  tol=0.0001)),
        ('сbr',  CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = 0.083,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          depth=7,
                          l2_leaf_reg=1)
        )
            ]

reg2 = StackingRegressor(estimators=estimators,
     final_estimator = KNeighborsRegressor(algorithm='auto', leaf_size=40, n_neighbors=7, weights='distance', n_jobs=-1)
                    )

    
#reg2.fit(X, y)

#predict = np.round((np.exp(reg2.predict(X))-1)/1000) *1000
#y_test_exp = np.exp(y)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 17.49%

3 вариант

In [ ]:
estimators=[
        ('rf1', RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=200, min_samples_split=2, min_samples_leaf=2, 
                            max_features='auto', max_depth=5, bootstrap=True, n_jobs=-1)),
        ('knr', KNeighborsRegressor(algorithm='auto', leaf_size=40, n_neighbors=7, weights='distance', n_jobs=-1)),
        ('сbr',  CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = 0.083,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          depth=7,
                          l2_leaf_reg=1)
        )
            ]

reg3 = StackingRegressor(estimators=estimators,
     final_estimator = GradientBoostingRegressor(n_estimators=300, random_state=RANDOM_SEED,  warm_start=10,  tol=0.0001)
                    )

    
reg3.fit(X, y)

predict = np.round((np.exp(reg3.predict(X))-1)/1000) *1000
y_test_exp = np.exp(y)-1

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test_exp, predict))*100:0.2f}%")

### **Точность модели по метрике MAPE: 12.50%**

4 вариант

In [ ]:
estimators=[
            ('сbr',  CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = 0.083,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          depth=7,
                          l2_leaf_reg=1)
        )
            ]

reg4 = StackingRegressor(estimators=estimators,
     final_estimator = GradientBoostingRegressor(n_estimators=300, random_state=RANDOM_SEED,  warm_start=10,  tol=0.0001)
                    )

    
#reg4.fit(X, y)

#predict = np.round((np.exp(reg4.predict(X))-1)/1000) *1000
#y_test_exp = np.exp(y)-1

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test_exp, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 12.66%

Видим, что основными алгоритмами в стекинге являются CatBoost и GradientBoosting - использование других алгоритмов дает незначительное улучшение модели 

По результатам стекинга видно, что так как CatBoost чувствителен к набору признаков,а так же его оптимальные параметры подобраны для обучения на исходном датасете, без дополнительных признаков, полученных от других алгоритмов, то его лучше ставить в базовый набор алгоритмов стекинга.

Финальным (итоговым) алгоритмом стекинга лучше всего ставить бустинг. В данном случае -  GradientBoosting

# Submission

Вычислим submission на основе 3-ой модели стекинга

In [ ]:
predict_submission = np.round((np.exp(reg3.predict(X_sub))-1)/1000) *1000
predict_submission 

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

* На лидерборде получили 15.24% Видно что стекинг в целом дает хорошие результаты, но сказываются некоторые различия между тренировочным и тестовым датасетом с каггла: возможно за полтора месяца (разница в собирании датасетов) произошло некоторое изменение цены автомобилей?

# Выводы
В итоге видим:


* Попытка удаления выбросов в тренировочном датасете метрику не улучшило
* Добавление новых бинарных признаков несколько ухудшает метрику, но ее улучшает введение полиномиальных
* Логарифмирование числовых и целевого признаков значительно улучшает метрику
* Поиск оптимальных параметров CatBoost незначительно улучшает метрику
* По результатам стекинга видно, что так как CatBoost чувствителен к набору признаков,а так же его оптимальные параметры подобраны для обучения на исходном датасете, без дополнительных признаков, полученных от других алгоритмов, то его лучше ставить в базовый набор алгоритмов стекинга.
* Финальным (итоговым) аглгоритмом стекинга лучше всего ставить бустинг. В данном случае -  GradientBoosting

Почему-то наилучший результат  на тестовом датасете (из каггла) показало использование только CatBoost с оптимальными параметрами. На тренировочном же датасете лучший результат показывает стекинг, с использованием CatBoost, GradientBoosting, RandomForest и  KNeighborsRegressor